<a href="https://colab.research.google.com/github/loaiwalid07/main/blob/main/Food_Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Food Recommendation system**

By: Loai Nazeer

In [10]:
# import modules
import pandas as pd
import matplotlib as plt
import numpy as np
import nltk
import gdown
import re
import pandas_profiling

In [5]:
# Download the datset from google drive
#https://drive.google.com/file/d/1pscWtFjiwVJXwuKUC9dv4kdLu7_FvJgV/view?usp=sharing
url = "https://drive.google.com/uc?id=1pscWtFjiwVJXwuKUC9dv4kdLu7_FvJgV"
data = gdown.download(url=url,output="food_data.csv",quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1pscWtFjiwVJXwuKUC9dv4kdLu7_FvJgV
To: /content/food_data.csv
100%|██████████| 109k/109k [00:00<00:00, 42.0MB/s]


In [7]:
#read the data in dataframe
df = pd.read_csv("food_data.csv")
df.head(5)

,Dish,Ingredients,URL,Class,Cuisine
0,BUTTERNUT SQUASH RIGATONI,"'rigatoni', 'butternut squash', 'parmesan', 's...",http://www.mobkitchen.co.uk/recipes/butternut-...,2,Indian
1,HASSELBACK BUTTERNUT SQUASH,"'butternut squash', 'chilli', 'flaked almonds'...",http://www.mobkitchen.co.uk/recipes/hasselback...,2,Indian
2,MOB'S TARTIFLETTE,"'potatoes', 'onions', 'cloves garlic', 'pancet...",http://www.mobkitchen.co.uk/recipes/mobs-tarti...,2,Indian
3,ROASTED ROOT VEGETABLE CURRY,"'parsnips', 'carrots', 'sweet potatoes', 'curr...",http://www.mobkitchen.co.uk/recipes/roasted-ro...,4,Italian
4,MOB'S BUFFALO WINGS,"'chicken wings', 'baking powder', 'mature ched...",http://www.mobkitchen.co.uk/recipes/buffalo-wings,2,Indian


In [19]:
for i in range(0,len(df["Ingredients"])):
  df["Ingredients"][i] =  re.sub(r'\'','',df["Ingredients"][i])
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Dish,Ingredients,URL,Class,Cuisine
0,BUTTERNUT SQUASH RIGATONI,"rigatoni, butternut squash, parmesan, shallots...",http://www.mobkitchen.co.uk/recipes/butternut-...,2,Indian
1,HASSELBACK BUTTERNUT SQUASH,"butternut squash, chilli, flaked almonds, feta...",http://www.mobkitchen.co.uk/recipes/hasselback...,2,Indian
2,MOB'S TARTIFLETTE,"potatoes, onions, cloves garlic, pancetta, whi...",http://www.mobkitchen.co.uk/recipes/mobs-tarti...,2,Indian
3,ROASTED ROOT VEGETABLE CURRY,"parsnips, carrots, sweet potatoes, curry powde...",http://www.mobkitchen.co.uk/recipes/roasted-ro...,4,Italian
4,MOB'S BUFFALO WINGS,"chicken wings, baking powder, mature cheddar c...",http://www.mobkitchen.co.uk/recipes/buffalo-wings,2,Indian
...,...,...,...,...,...
407,KATY'S GUACAMOLE COURGETTI,"avocados, chillies, coriander, limes, courgett...",http://www.mobkitchen.co.uk/recipes/katys-guac...,0,Chinese
408,FETA & OLIVE STUFFED ROAST PEPPERS,"cherry tomatoes, chilli, peppers red orange gr...",http://www.mobkitchen.co.uk/recipes/feta-olive...,1,French
409,COURGETTE PARMAGIANA,"tomatoes, black olives, balls mozzarella, garl...",http://www.mobkitchen.co.uk/recipes/courgette-...,1,French
410,STICKY SESAME & GINGER TOFU WITH PAK CHOI,"chilli, spring onions, pak choi, ginger, tofu,...",http://www.mobkitchen.co.uk/recipes/sticky-ses...,0,Chinese


In [26]:
feature = pd.DataFrame([df['Ingredients'],df["Class"]],index=None).T

,Ingredients,Class
0,"rigatoni, butternut squash, parmesan, shallots...",2
1,"butternut squash, chilli, flaked almonds, feta...",2
2,"potatoes, onions, cloves garlic, pancetta, whi...",2
3,"parsnips, carrots, sweet potatoes, curry powde...",4
4,"chicken wings, baking powder, mature cheddar c...",2
...,...,...
407,"avocados, chillies, coriander, limes, courgett...",0
408,"cherry tomatoes, chilli, peppers red orange gr...",1
409,"tomatoes, black olives, balls mozzarella, garl...",1
410,"chilli, spring onions, pak choi, ginger, tofu,...",0


In [29]:
#TF-IDF transform
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(feature['Ingredients']).todense()
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tfidf = tf_transformer.transform(X_train_counts).todense()
print("Data dimensions with TFIDF and BOW:",X_train_tfidf.shape)

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = linear_kernel(X_train_tfidf, X_train_tfidf)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [31]:
df = df.reset_index()
dishs = df['Dish']
indices = pd.Series(df.index, index=df['Dish'])

In [34]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return dishs.iloc[movie_indices]

In [37]:
get_recommendations("MOB'S BUFFALO WINGS").head(10)

216                   CHEESY PANCETTA POTATOES
101                          BBQ CHICKEN WINGS
263    CHICKEN, MUSHROOM & CHEDDAR MONSTER PIE
213                   ASIAN CHICKEN DRUMSTICKS
36                         SPICY CHICKEN KATSU
191                  MOB'S CHORIZO QUESADILLAS
275        DOUBLE BACON CHIPOTLE BAGEL BURGERS
73           CASA DO FRANGO'S RICE FOR THE MOB
186                    EGGY GUACAMOLE CRUMPETS
121                MOB'S PERI PERI CRUNCH WRAP
Name: Dish, dtype: object